In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
from iris_plant_visualizer import IrisPlantVisualizer
from iris_plant_animation_visualizer import IrisPlantVisualizer as viz2

import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import visualization_utils as viz_utils


In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions

In [4]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint.Click on the two links at the bottom to view the plant and the configuration space.

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))
locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = np.array([-val, -val])
q_high = np.array([val, val])
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)
cspace_frame = RigidTransform(RotationMatrix.MakeYRotation(np.pi/2)@RotationMatrix.MakeZRotation(np.pi/2), np.array([0,3,1]))
# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_polytope, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, 
                                          num_points = 100)
# visualizer.meshcat.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7001
INFO:drake:Meshcat listening for connections at http://localhost:7002


In [6]:
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [7]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0', step = 0.02))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1', step = 0.02))

q = np.array([1.70, 0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7, step=0.02)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7, step=0.02)

# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [8]:
# Some seedpoints
seed_points_q = np.array([   
                              [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.5,-1.08],
                              [-1.06, 0.48]
                              ])

seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
start = seed_points[0]
end = seed_points[-1]

visualizer.plot_cspace_points(seed_points, "/iris_seed_points", radius = 0.02)
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region. Chosen for color-blind compatibility
default_alpha = 0.2
colors_dict = {
    0: Rgba(0.565, 0.565, 0.565, default_alpha), # gray
    1: Rgba(0.118, 0.533, 0.898, default_alpha), # bluish
    2: Rgba(1,     0.757, 0.027, default_alpha), # gold
    3: Rgba(0,     0.549, 0.024, default_alpha), # green   
    4: Rgba(0.055, 0.914, 0.929, default_alpha), # teal 
}

initial_regions = [(make_default_polytope_at_point(s), colors_dict[i]) for i, s in enumerate(seed_points)]
# visualizer.add_group_of_regions_to_visualization(initial_regions, "initial_regions",
#                             wireframe = False,
#                            opacity = default_alpha)
visualizer.show_res_q(q_star)

In [9]:
visualizer.meshcat_cspace.Delete("/s")


In [10]:
visualizer._plane_indices_of_interest

[]

## First we set up some options for our different certification modes

In [11]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = False

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-8


bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 50
bilinear_alternation_options.convergence_tol = 1e-3
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 50
binary_search_options.max_iter = 100

Clim = np.vstack([np.eye(q_star.shape[0]), -np.eye(q_star.shape[0])])
dlim = np.concatenate([Ratfk.ComputeSValue(q_high, q_star), -Ratfk.ComputeSValue(q_low, q_star)])
def polytope_with_joint_limits(C, d):
    return HPolyhedron(np.vstack([C, Clim]), np.concatenate([d, dlim]))

In [12]:
from time import perf_counter
bilinear_alternation_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
times_per_region = []
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.1) # make the above print before the Drake log.
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    t_start = time.perf_counter()
    result = cspace_free_polytope.SearchWithBilinearAlternation(set(), initial_region.A(),
                                                                initial_region.b(),
                                                                bilinear_alternation_options)
    t_end = time.perf_counter()
    times_per_region.append(t_end - t_start)
    bilinear_alternation_results_by_seed_point[tuple(s)] = [(polytope_with_joint_limits(cert.C, cert.d),
                                                             cert, color) for cert in result]
    

INFO:drake:det(Q) at the beginning is 4.900500382473016e-05


starting seedpoint 1/5


INFO:drake:Iteration 0: det(Q)=0.00014576238925864208
INFO:drake:Iteration 1: det(Q)=0.0002806159113297737
INFO:drake:Iteration 2: det(Q)=0.0005344000732312085
INFO:drake:Iteration 3: det(Q)=0.0009169467132802288
INFO:drake:Iteration 4: det(Q)=0.0013848549945793937
INFO:drake:Iteration 5: det(Q)=0.0019126336178197192
INFO:drake:Iteration 6: det(Q)=0.00255550225428644
INFO:drake:Iteration 7: det(Q)=0.003337093699566204
INFO:drake:Iteration 8: det(Q)=0.00430669080783459
INFO:drake:Iteration 9: det(Q)=0.005568220382883087
INFO:drake:Iteration 10: det(Q)=0.007114884752347183
INFO:drake:Iteration 11: det(Q)=0.008935629008910621
INFO:drake:Iteration 12: det(Q)=0.011160505913197317
INFO:drake:Iteration 13: det(Q)=0.013856731449179946
INFO:drake:Iteration 14: det(Q)=0.016859548786580456
INFO:drake:Iteration 15: det(Q)=0.020610878066453044
INFO:drake:Iteration 16: det(Q)=0.02530337015678481
INFO:drake:Iteration 17: det(Q)=0.02861864786435216
INFO:drake:Iteration 18: det(Q)=0.031073885939933994


starting seedpoint 2/5


INFO:drake:Iteration 0: det(Q)=0.00014131240847090125
INFO:drake:Iteration 1: det(Q)=0.00036805961026473594
INFO:drake:Iteration 2: det(Q)=0.0007327701038891825
INFO:drake:Iteration 3: det(Q)=0.001456733282687195
INFO:drake:Iteration 4: det(Q)=0.002961332348494822
INFO:drake:Iteration 5: det(Q)=0.005965834368885852
INFO:drake:Iteration 6: det(Q)=0.009942655734023328
INFO:drake:Iteration 7: det(Q)=0.01235925129642312
INFO:drake:Iteration 8: det(Q)=0.013822141909334801
INFO:drake:Iteration 9: det(Q)=0.015100253158656518
INFO:drake:Iteration 10: det(Q)=0.01629489374179905
INFO:drake:Iteration 11: det(Q)=0.01721702132606201
INFO:drake:Iteration 12: det(Q)=0.018057449774436303
INFO:drake:Iteration 13: det(Q)=0.018789068669064324
INFO:drake:Iteration 14: det(Q)=0.019457449285388582
INFO:drake:Iteration 15: det(Q)=0.019972553730983292
INFO:drake:Iteration 16: det(Q)=0.020319434371489833
INFO:drake:Iteration 17: det(Q)=0.0206264546237852
INFO:drake:Iteration 18: det(Q)=0.020905526613063476
INF

...

INFO:drake:Iteration 0: det(Q)=0.0002893803632078695
INFO:drake:Iteration 1: det(Q)=0.000629240698708994
INFO:drake:Iteration 2: det(Q)=0.0010005786406084907
INFO:drake:Iteration 3: det(Q)=0.0015026235141849568
INFO:drake:Iteration 4: det(Q)=0.0022000574340110984
INFO:drake:Iteration 5: det(Q)=0.0030720890177413714
INFO:drake:Iteration 6: det(Q)=0.004162115304839216
INFO:drake:Iteration 7: det(Q)=0.005547234537234709
INFO:drake:Iteration 8: det(Q)=0.0074292604396001775
INFO:drake:Iteration 9: det(Q)=0.010085315083395479
INFO:drake:Iteration 10: det(Q)=0.013281577185173903
INFO:drake:Iteration 11: det(Q)=0.01694537902529072
INFO:drake:Iteration 12: det(Q)=0.02153020666681398
INFO:drake:Iteration 13: det(Q)=0.027388154267308482
INFO:drake:Iteration 14: det(Q)=0.035545598544039415
INFO:drake:Iteration 15: det(Q)=0.04730253089744523
INFO:drake:Iteration 16: det(Q)=0.06925458037467991
INFO:drake:Iteration 17: det(Q)=0.0933919784520877
INFO:drake:Iteration 18: det(Q)=0.10877284559474545
INFO

starting seedpoint 4/5


INFO:drake:Iteration 0: det(Q)=0.00010339887932332229
INFO:drake:Iteration 1: det(Q)=0.00018579145628959672
INFO:drake:Iteration 2: det(Q)=0.0002769318578043369
INFO:drake:Iteration 3: det(Q)=0.0004675590586949072
INFO:drake:Iteration 4: det(Q)=0.0007918939209971374
INFO:drake:Iteration 5: det(Q)=0.0011657905502829118
INFO:drake:Iteration 6: det(Q)=0.0017386046717462675
INFO:drake:Iteration 7: det(Q)=0.002541565057009815
INFO:drake:Iteration 8: det(Q)=0.0034983439448962165
INFO:drake:Iteration 9: det(Q)=0.0046491207103816845
INFO:drake:Iteration 10: det(Q)=0.006070204334967574
INFO:drake:Iteration 11: det(Q)=0.007643903525013378
INFO:drake:Iteration 12: det(Q)=0.009792904967279723
INFO:drake:Iteration 13: det(Q)=0.01334431554136183
INFO:drake:Iteration 14: det(Q)=0.018856065597072914
INFO:drake:Iteration 15: det(Q)=0.02782325429042248
INFO:drake:Iteration 16: det(Q)=0.044361018913048784
INFO:drake:Iteration 17: det(Q)=0.057292038884133555
INFO:drake:Iteration 18: det(Q)=0.0682209066819

starting seedpoint 5/5


INFO:drake:Iteration 0: det(Q)=9.187667490570257e-05
INFO:drake:Iteration 1: det(Q)=0.00015239078399186516
INFO:drake:Iteration 2: det(Q)=0.00023425033830250453
INFO:drake:Iteration 3: det(Q)=0.0003362913456625858
INFO:drake:Iteration 4: det(Q)=0.00047180029306225185
INFO:drake:Iteration 5: det(Q)=0.000631131513203011
INFO:drake:Iteration 6: det(Q)=0.0008448350871849316
INFO:drake:Iteration 7: det(Q)=0.0011059142547443187
INFO:drake:Iteration 8: det(Q)=0.0014591068950592092
INFO:drake:Iteration 9: det(Q)=0.001984403557036214
INFO:drake:Iteration 10: det(Q)=0.0027109588797051883
INFO:drake:Iteration 11: det(Q)=0.003299000202382135
INFO:drake:Iteration 12: det(Q)=0.004479237882434005
INFO:drake:Iteration 13: det(Q)=0.005494824570979182
INFO:drake:Iteration 14: det(Q)=0.007830253356251222
INFO:drake:Iteration 15: det(Q)=0.011396510162193785
INFO:drake:Iteration 16: det(Q)=0.017585510310220712
INFO:drake:Iteration 17: det(Q)=0.024454094550338586
INFO:drake:Iteration 18: det(Q)=0.0329432871

In [13]:
max_frames = max((len(item) for item in bilinear_alternation_results_by_seed_point.values()))


In [14]:
print(max_frames)

50


In [15]:
regions_per_frame = {i: list() for i in range(max_frames)}
for _, v in bilinear_alternation_results_by_seed_point.items():
    for i in range(max_frames):
        res = v[min(i, len(v)-1)]
        regions_per_frame[i].append((res[0], res[2]))
final_regions = regions_per_frame[max_frames-1]


In [16]:
for i, regions in regions_per_frame.items():
    print(regions)
    visualizer.add_group_of_regions_to_visualization(regions, f"/frame_{i}",
                                                    fill = False)
    visualizer.meshcat_cspace.SetProperty(f"/frame_{i}", 'visible', False)


[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f678825d9b0>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788270bf0>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788270630>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828fb70>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418e670>, Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]
[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f67882638f0>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f67882704b0>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828a3f0>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788291c30>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2))

[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788263b30>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788270fb0>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f67882913b0>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828ff30>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418ecf0>, Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]
[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f678826c030>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828a030>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f67882913f0>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828ff70>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2))

[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f678826c530>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828a370>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788291770>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418d330>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418f470>, Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]
[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f678826c8f0>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828a370>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f67882917b0>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418d370>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2))

[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f67882700f0>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828a370>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788291b30>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418d8f0>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418f6f0>, Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]
[(<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788270070>, Rgba(r=0.565, g=0.565, b=0.565, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678828a370>, Rgba(r=0.118, g=0.533, b=0.898, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f6788291b70>, Rgba(r=1.0, g=0.757, b=0.027, a=0.2)), (<pydrake.geometry.optimization.HPolyhedron object at 0x7f678418d970>, Rgba(r=0.0, g=0.549, b=0.024, a=0.2))

In [17]:
visualizer.add_group_of_regions_to_visualization(final_regions, f"/final_regions",
                                                fill = 0.2,
                                                line_width = 1e-3,
                                                wireframe = False,
                                                resolution = 50)

In [18]:
# visualize growth
dt_frame = 0.5
visualizer.visualizer_cspace.StartRecording()
for i in range(max_frames):
    visualizer.cspace_diagram_context.SetTime(i * dt_frame)
    visualizer.cspace_diagram.ForcedPublish(visualizer.cspace_diagram_context)    
visualizer.visualizer_cspace.StopRecording()
time_points = np.linspace(0, dt_frame * max_frames, max_frames)

In [19]:
animation = visualizer.visualizer_cspace.get_mutable_recording()
for i in range(max_frames):
    animation.SetProperty(0, f"/frame_{i}", "visible", False)
    
for i in range(max_frames):
    frame = animation.frame(time_points[i])
    if i > 0:
        animation.SetProperty(frame, f"/frame_{i-1}", "visible", False)
    animation.SetProperty(frame, f"/frame_{i}", "visible", True)
final_frame = animation.frame(dt_frame * max_frames)
# animation.SetProperty(final_frame, f"/frame_{max_frames-1}", "visible", False)
    
visualizer.visualizer_cspace.PublishRecording()


In [20]:
with open("pinball_growth.html", "w") as f:
    f.write(visualizer.meshcat_cspace.StaticHtml())
    